In [294]:
import pandas as pd
from collections import Counter, namedtuple
import urllib.request
from razdel import tokenize, sentenize
from string import punctuation
from tqdm import tqdm
import csv
import fasttext
import numpy as np
import itertools
from transformers import AutoTokenizer
from sklearn.linear_model import LogisticRegression
import random
import torch
from torch import nn, optim
import torch.nn.functional as F
import time

# Домашнее задание
## Yes/No Questions

В этом домашнем задании вы будете работать с корпусом BoolQ. Корпус состоит из вопросов, предполагающих бинарный ответ (да / нет), абзацев из Википедии,  содержащих ответ на вопрос, заголовка статьи, из которой извлечен абзац и непосредственно ответа (true / false).

Корпус описан в статье:

Christopher Clark, Kenton Lee, Ming-Wei Chang, Tom Kwiatkowski, Michael Collins, Kristina Toutanova
BoolQ: Exploring the Surprising Difficulty of Natural Yes/No Questions

https://arxiv.org/abs/1905.10044


Корпус (train-dev split) доступен в репозитории проекта:  https://github.com/google-research-datasets/boolean-questions

Используйте для обучения train часть корпуса, для валидации и тестирования – dev часть. 

Каждый бонус пункт оцениватся в 1 балл. 

### Пример вопроса: 
question: is batman and robin a sequel to batman forever

title: Batman & Robin (film)

answer: true

passage: With the box office success of Batman Forever in June 1995, Warner Bros. immediately commissioned a sequel. They hired director Joel Schumacher and writer Akiva Goldsman to reprise their duties the following August, and decided it was best to fast track production for a June 1997 target release date, which is a break from the usual 3-year gap between films. Schumacher wanted to homage both the broad camp style of the 1960s television series and the work of Dick Sprang. The storyline of Batman & Robin was conceived by Schumacher and Goldsman during pre-production on A Time to Kill. Portions of Mr. Freeze's back-story were based on the Batman: The Animated Series episode ''Heart of Ice'', written by Paul Dini.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 2-х человек.
2. Домашнее задание оформляется в виде отчета в ipython-тетрадке. 
3. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.

## Часть 1. [1 балл] Эксплоративный анализ
1. Посчитайте долю yes и no классов в корпусе
2. Оцените среднюю длину вопроса
3. Оцените среднюю длину параграфа
4. Предположите, по каким эвристикам были собраны вопросы (или найдите ответ в статье). Продемонстриуйте, как эти эвристики повлияли на структуру корпуса. 

##### загрузка данных

In [2]:
train_data_df = pd.read_json('train.jsonl', lines=True, orient='records')
dev_data_df = pd.read_json('dev.jsonl', lines=True, orient='records')
train_data_df.head(3)

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...


##### 1)

In [15]:
a_1 = train_data_df.answer.value_counts()\
                   .reset_index()\
                   .rename(columns={'index':'answer', 'answer':'cnt'})
a_1['rt'] = a_1['cnt']/a_1['cnt'].sum()
a_1['rt'] = a_1['rt'].round(2)
a_1

,answer,cnt,rt
0,True,5874,0.62
1,False,3553,0.38


##### 2)

In [21]:
train_data_df.question.apply(len).mean().round(0)

44.0

##### 3)

In [22]:
train_data_df.passage.apply(len).mean().round(0)

566.0

##### 4) <br>
вопросы начинаются со слов-индикаторов

In [41]:
a_4 = train_data_df.question.apply(lambda x: x.split()[0])\
                            .value_counts()\
                            .reset_index()\
                            .rename(columns={'index':'1st_qst_word','question':'cnt'})
a_4 = a_4[a_4.cnt>5]
a_4['rt'] = (a_4.cnt / len(train_data_df)).round(2)
a_4

,1st_qst_word,cnt,rt
0,is,4190,0.44
1,can,1136,0.12
2,does,952,0.10
3,are,693,0.07
4,do,664,0.07
5,did,461,0.05
6,was,335,0.04
7,has,302,0.03
8,will,181,0.02
9,the,91,0.01


## Часть 2. [1 балл] Baseline
1. Оцените accuracy точность совсем простого базового решения: присвоить каждой паре вопрос-ответ в dev части самый частый класс из train части
2. Оцените accuracy чуть более сложного базового решения: fasttext на текстах, состоящих из склееных вопросов и абзацев (' '.join([question, passage]))

Почему fasttext плохо справляется с этой задачей?

##### 1)

In [69]:
most_frq_ans = Counter(train_data_df.answer).most_common(1)[0][0]
(dev_data_df.answer == most_frq_ans).sum() / len(dev_data_df)

0.6217125382262997

##### 2)

https://towardsdatascience.com/fasttext-for-text-classification-a4b38cbff27c

preprocessing

In [144]:
# train
train_text = [  ' '.join([question, passage])
                for question, passage 
                in zip(train_data_df.question.tolist(), train_data_df.passage.tolist())]

for i in tqdm(range(len(train_text))):
    text_i = []
    for sent in sentenize(train_text[i]):
        sent_i = [ j.text.lower() for j in tokenize(sent.text) if j.text not in list(punctuation)]
        text_i.extend(sent_i)
    train_text[i] = text_i
    
# dev
dev_text = [  ' '.join([question, passage])
                for question, passage 
                in zip(dev_data_df.question.tolist(), dev_data_df.passage.tolist())]

for i in tqdm(range(len(dev_text))):
    text_i = []
    for sent in sentenize(dev_text[i]):
        sent_i = [ j.text.lower() for j in tokenize(sent.text) if j.text not in list(punctuation)]
        text_i.extend(sent_i)
    dev_text[i] = text_i

100%|█████████████████████████████████████| 3270/3270 [00:02<00:00, 1123.75it/s]


saving txt-files

In [46]:
# train
train_dataset = pd.DataFrame({'text': train_text, 'target': train_data_df.answer.tolist()})
train_dataset['text'] = train_dataset['text'].apply(lambda x: ' '.join(x))
train_dataset['target'] = train_dataset['target'].apply(lambda x: '__label__' + str(x))
train_dataset[['target', 'text']].to_csv('train.txt', 
                                         index = False, 
                                         sep = ' ',
                                         header = None, 
                                         quoting = csv.QUOTE_NONE, 
                                         quotechar = "", 
                                         escapechar = " ")
# dev
dev_dataset = pd.DataFrame({'text': dev_text, 'target': dev_data_df.answer.tolist()})
dev_dataset['text'] = dev_dataset['text'].apply(lambda x: ' '.join(x))
dev_dataset['target'] = dev_dataset['target'].apply(lambda x: '__label__' + str(x))
dev_dataset[['target', 'text']].to_csv('dev.txt', 
                                         index = False, 
                                         sep = ' ',
                                         header = None, 
                                         quoting = csv.QUOTE_NONE, 
                                         quotechar = "", 
                                         escapechar = " ")

training

In [47]:
model = fasttext.train_supervised('train.txt', wordNgrams = 2)

Read 0M words
Number of words:  47170
Number of labels: 2
Progress: 100.0% words/sec/thread:  995967 lr:  0.000000 avg.loss:  0.654642 ETA:   0h 0m 0s


test

In [49]:
model.test('dev.txt')   

(3270, 0.6290519877675841, 0.6290519877675841)

In [54]:
predict = model.predict(dev_dataset['text'].iloc[0])[0]
predict

('__label__True',)

## Часть 3. [1 балл] Используем эмбеддинги предложений
1. Постройте BERT эмбеддинги вопроса и абзаца. Обучите логистическую регрессию на конкатенированных эмбеддингах вопроса и абзаца и оцените accuracy этого решения. 

[bonus] Используйте другие модели эмбеддингов, доступные, например, в библиотеке 🤗 Transformers. Какая модель эмбеддингов даст лучшие результаты?

[bonus] Предложите метод аугментации данных и продемонстрируйте его эффективность. 

##### 1) BERT embedding

In [93]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# train
train_dataset = train_data_df[['question', 'passage', 'answer']].copy()
train_dataset['question'] = train_dataset.question.apply(lambda x: tokenizer(x)['input_ids']) #tokenizer.tokenize(x))
train_dataset['question'] = train_dataset.question.apply(lambda x: x[:30] + [0]*(30-len(x[:30]))) 
train_dataset['passage'] = train_dataset.passage.apply(lambda x: tokenizer(x)['input_ids'])
train_dataset['passage'] = train_dataset.passage.apply(lambda x: x[:1000] + [0]*(1000-len(x[:1000])))
train_dataset['q&p'] = train_dataset[['question', 'passage']].apply(lambda x: x[0]+x[1], axis=1)

# dev
dev_dataset = dev_data_df[['question', 'passage', 'answer']].copy()
dev_dataset['question'] = dev_dataset.question.apply(lambda x: tokenizer(x)['input_ids'])
dev_dataset['question'] = dev_dataset.question.apply(lambda x: x[:30] + [0]*(30-len(x[:30]))) 
dev_dataset['passage'] = dev_dataset.passage.apply(lambda x: tokenizer(x)['input_ids'])
dev_dataset['passage'] = dev_dataset.passage.apply(lambda x: x[:1000] + [0]*(1000-len(x[:1000])))
dev_dataset['q&p'] = dev_dataset[['question', 'passage']].apply(lambda x: x[0]+x[1], axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (928 > 512). Running this sequence through the model will result in indexing errors


model

In [126]:
X_train, y_train = np.array(train_dataset['q&p'].tolist()), train_dataset['answer'].astype(int)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
predict = clf.predict(X_train)
print('train accuracy:', (predict == y_train).sum() / len(y_train))

train accuracy: 0.6375304975071603


/Users/16971921/.conda/envs/py38_venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


test

In [128]:
X_test, y_test = np.array(dev_dataset['q&p'].tolist()), dev_dataset['answer'].astype(int)
predict = clf.predict(X_test)
print('test accuracy:', (predict == y_test).sum() / len(y_test))

test accuracy: 0.6067278287461774


## Часть 3. [3 балла] DrQA-подобная архитектура

Основана на статье: Reading Wikipedia to Answer Open-Domain Questions

Danqi Chen, Adam Fisch, Jason Weston, Antoine Bordes

https://arxiv.org/abs/1704.00051

Архитектура DrQA предложена для задачи SQuAD, но легко может быть адаптирована к текущему заданию. Модель состоит из следующих блоков:
1. Кодировщик абзаца [paragraph encoding] – LSTM, получаящая на вход вектора слов, состоящие из: 
* эмбеддинга слова (w2v или fasttext)
* дополнительных признаков-индикаторов, кодирующих в виде one-hot векторов часть речи слова, является ли оно именованной сущностью или нет, встречается ли слово в вопросе или нет 
* выровненного эмбеддинга вопроса, получаемого с использованием soft attention между эмбеддингами слов из абзаца и эмбеддингом вопроса.

$f_{align}(p_i) = \sum_j􏰂 a_{i,j} E(q_j)$, где $E(q_j)$ – эмбеддинг слова из вопроса. Формула для $a_{i,j}$ приведена в статье. 

2. Кодировщик вопроса [question encoding] – LSTM, получаящая на вход эмбеддинги слов из вопроса. Выход кодировщика: $q = 􏰂\sum_j􏰂  b_j q_j$. Формула для $b_{j}$ приведена в статье. 

3. Слой предсказания. 

Предложите, как можно было модифицировать последний слой предсказания в архитектуре DrQA, с учетом того, что итоговое предсказание – это метка yes / no, предсказание которой проще, чем предсказание спана ответа для SQuAD.

Оцените качество этой модели для решения задачи. 

[bonus] Замените входные эмбеддинги и все дополнительные признаки, используемые кодировщиками, на BERT эмбеддинги. Улучшит ли это качество результатов?

##### 1)

get dataset + tokenize

In [3]:
Sample = namedtuple("Sample", "question, passage, labels")

def get_dataset(path_file: list):
    samples = []
    file = pd.read_json(path_file, lines=True, orient='records')
    question = file.question.tolist()
    passage = file.passage.tolist()
    labels = file.answer.astype(int)
    
    for i in tqdm(range(len(file))):
        #question
        question_i = []
        for sent in sentenize(question[i]):
            sent_i = [ j.text.lower() for j in tokenize(sent.text) if j.text not in list(punctuation)]
            question_i.extend(sent_i)
        
        #question
        passage_i = []
        for sent in sentenize(passage[i]):
            sent_i = [ j.text.lower() for j in tokenize(sent.text) if j.text not in list(punctuation)]
            passage_i.extend(sent_i)
    
        sample = Sample(question_i, passage_i, labels[i])
        samples.append(sample)

    return samples

train val split

In [4]:
data = get_dataset('train.jsonl')
random.shuffle(data)

train_size = int(len(data)*0.8)
train = data[:train_size]
val = data[train_size:]
test = get_dataset('dev.jsonl')

100%|█████████████████████████████████████| 3270/3270 [00:02<00:00, 1124.04it/s]


word dict

In [5]:
word_set = list( {token for sample in train for token in sample.question} |
                 {token for sample in train for token in sample.passage}
               )
word_set.insert(0, '<unk>'), word_set.insert(0, '<pad>')
len(word_set)

42113

batch generation

In [133]:
a = [4,23,46,21367,7,234,7,4,1,34,4]
np.where(np.array(a) == 4)

(array([ 0,  7, 10]),)

In [233]:
def get_next_gen_batch(samples, max_qst_len=50, max_psg_len=512, batch_size=32):
    indices = np.arange(len(samples))
    np.random.shuffle(indices)
    batch_begin = 0
    with tqdm(total=len(samples)) as pbar:
        while batch_begin < len(samples):
            batch_indices = indices[batch_begin: batch_begin + 32]
            batch_qst = []
            batch_psg = []
            batch_psg_fts = []
            batch_labels = []
            batch_qst_mask = torch.ByteTensor(len(batch_indices), max_qst_len).fill_(1)
            batch_psg_mask = torch.ByteTensor(len(batch_indices), max_psg_len).fill_(1)
            for data_ind in batch_indices:
                ind = list(batch_indices).index(data_ind)
                
                sample = samples[data_ind]
                #вопрос
                question = torch.zeros(max_qst_len, dtype=torch.long) #.cuda()
                for token_num, token in enumerate(sample.question[:max_qst_len]):
                    question[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                question_len = len(sample.question[:max_qst_len])
                batch_qst_mask[ind, :question_len].fill_(0)
                #параграф
                passage = torch.zeros(max_psg_len, dtype=torch.long) #.cuda()
                passage_features = torch.zeros(max_psg_len, max_qst_len, dtype=torch.long)
                for token_num, token in enumerate(sample.passage[:max_psg_len]):
                    passage[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                    q_inds = np.where(np.array(sample.question[:max_qst_len]) == token)
                    for q_ind in q_inds:
                        passage_features[token_num, q_ind] = 1
                passage_len = len(sample.passage[:max_psg_len])
                batch_psg_mask[ind, :passage_len].fill_(0)
                    
                labels = sample.labels

                batch_qst.append(question)
                batch_psg.append(passage)
                batch_psg_fts.append(passage_features)
                batch_labels.append(labels)
              
            batch_begin += batch_size
            pbar.update(batch_size)
          
            batch_qst = torch.stack(batch_qst)
            batch_psg = torch.stack(batch_psg)
            batch_psg_fts = torch.stack(batch_psg_fts)
            batch_labels = torch.LongTensor(batch_labels)
            yield batch_indices, batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask, batch_labels

In [135]:
for _, batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask, batch_labels in get_next_gen_batch(train):
    1+1
    break
print()
print( 'batch_qst'.ljust(20), batch_qst.size() )
print( 'batch_qst_mask'.ljust(20), batch_qst_mask.size() )
print( 'batch_psg'.ljust(20), batch_psg.size() )
print( 'batch_psg_fts'.ljust(20), batch_psg_fts.size() )
print( 'passage_mask'.ljust(20), batch_psg_mask.size() )
print( 'batch_labels'.ljust(20), batch_labels.size() )

  0%|▏                                        | 32/7541 [00:05<20:38,  6.07it/s]


batch_qst            torch.Size([32, 50])
batch_qst_mask       torch.Size([32, 50])
batch_psg            torch.Size([32, 512])
batch_psg_fts        torch.Size([32, 512, 50])
passage_mask         torch.Size([32, 512])
batch_labels         torch.Size([32])


model

In [287]:
# https://github.com/facebookresearch/DrQA/tree/main
class SeqAttnMatch(nn.Module):
    """Given sequences X and Y, match sequence Y to each element in X.

    * o_i = sum(alpha_j * y_j) for i in X
    * alpha_j = softmax(y_j * x_i)
    """

    def __init__(self, input_size, identity=False):
        super(SeqAttnMatch, self).__init__()
        if not identity:
            self.linear = nn.Linear(input_size, input_size)
        else:
            self.linear = None

    def forward(self, x, y, y_mask):
        """
        Args:
            x: batch * len1 * hdim
            y: batch * len2 * hdim
            y_mask: batch * len2 (1 for padding, 0 for true)
        Output:
            matched_seq: batch * len1 * hdim
        """
        # Project vectors
        if self.linear:
            x_proj = self.linear(x.view(-1, x.size(2))).view(x.size())
            x_proj = F.relu(x_proj)
            y_proj = self.linear(y.view(-1, y.size(2))).view(y.size())
            y_proj = F.relu(y_proj)
        else:
            x_proj = x
            y_proj = y

        # Compute scores
        scores = x_proj.bmm(y_proj.transpose(2, 1))

        # Mask padding
        y_mask = y_mask.unsqueeze(1).expand(scores.size())
        scores.data.masked_fill_(y_mask.data, -float('inf'))

        # Normalize with softmax
        alpha_flat = F.softmax(scores.view(-1, y.size(1)), dim=-1)
        alpha = alpha_flat.view(-1, x.size(1), y.size(1))

        # Take weighted average
        matched_seq = alpha.bmm(y)
        return matched_seq

class LinearSeqAttn(nn.Module):
    """Self attention over a sequence:

    * o_i = softmax(Wx_i) for x_i in X.
    """

    def __init__(self, input_size):
        super(LinearSeqAttn, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x, x_mask):
        """
        Args:
            x: batch * len * hdim
            x_mask: batch * len (1 for padding, 0 for true)
        Output:
            alpha: batch * len
        """
        x_flat = x.view(-1, x.size(-1))
        scores = self.linear(x_flat).view(x.size(0), x.size(1))
        scores.data.masked_fill_(x_mask.data, -float('inf'))
        alpha = F.softmax(scores, dim=-1)
        return alpha
    
#------------------------------------------------------------------------------------------
    
class DrQA_model(nn.Module):
    def __init__(self,
                 word_set_size,
                 question_max_size = 50,
                 passage_max_size = 512,
                 word_embedding_dim=128,
                 lstm_embedding_dim = 24,
                 classes_count=2):
        super().__init__()
        
        self.qemb_match = SeqAttnMatch(word_embedding_dim)
        self.self_attn = LinearSeqAttn(2*lstm_embedding_dim)
        self.embedding = nn.Embedding(word_set_size, word_embedding_dim)
        self.psg_rnn = nn.LSTM(2*word_embedding_dim+question_max_size, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.qst_rnn = nn.LSTM(word_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.lin = nn.Linear(passage_max_size+1, classes_count)

    def forward(self, questions, question_mask, passages, passage_features, passage_mask):
        #эмбеддинги слов w2v
        q_emb = self.embedding(questions)
        p_emb = self.embedding(passages)
        # выровненный эмбеддинг вопроса, получаемый с использованием 
        # soft attention между эмбеддингами слов из абзаца и эмбеддингом вопроса
        q_weighted_emb = self.qemb_match(p_emb, q_emb, question_mask)
        # дополнительных признаков-индикаторов, кодирующих в виде one-hot 
        # векторов встречается ли слово в вопросе или нет
        drnn_input = [p_emb]
        drnn_input.append(q_weighted_emb)
        drnn_input.append(passage_features)
        drnn_input = torch.cat(drnn_input, 2)
        
        # 1) paragraph encoding (LSTM)
        #    packs a Tensor containing padded sequences of variable length.
        #    compute sorted sequence lengths
        lengths = passage_mask.data.eq(0).long().sum(1)
        _, idx_sort = torch.sort(lengths, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)
        lengths = list(lengths[idx_sort])
        #    ---
        p_rnn_input = drnn_input.index_select(0, idx_sort) # sort x
        p_rnn_input = nn.utils.rnn.pack_padded_sequence(p_rnn_input, lengths, batch_first=True) # pack it up
        #    ---
        p_hidden, (final_hidden_state, final_cell_state) = self.psg_rnn(p_rnn_input) #LSTM
        p_output = nn.utils.rnn.pad_packed_sequence(p_hidden, batch_first=True)[0] # unpack 
        p_output = p_output.index_select(0, idx_unsort) # unsort
        #    pad up to original batch sequence length
        if p_output.size(1) != passage_mask.size(1):
            padding = torch.zeros(p_output.size(0), 
                                  passage_mask.size(1) - p_output.size(1),
                                  p_output.size(2)).type(p_output.data.type())
            p_output = torch.cat([p_output, padding], 1)
        
        # 2) question encoding (LSTM)
        lengths = question_mask.data.eq(0).long().sum(1)
        _, idx_sort = torch.sort(lengths, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)
        lengths = list(lengths[idx_sort])
        #    ---
        q_rnn_input = q_emb.index_select(0, idx_sort) # sort x
        q_rnn_input = nn.utils.rnn.pack_padded_sequence(q_rnn_input, lengths, batch_first=True) # pack it up
        #    ---
        q_hidden, (final_hidden_state, final_cell_state) = self.qst_rnn(q_rnn_input) #LSTM
        q_hidden = nn.utils.rnn.pad_packed_sequence(q_hidden, batch_first=True)[0]   # unpack 
        q_hidden = q_hidden.index_select(0, idx_unsort) # unsort
        #    pad up to original batch sequence length
        if q_hidden.size(1) != question_mask.size(1):
            padding = torch.zeros(q_hidden.size(0), 
                                  question_mask.size(1) - q_hidden.size(1),
                                  q_hidden.size(2)).type(q_hidden.data.type())
            q_hidden = torch.cat([q_hidden, padding], 1)
        #    взвешенное представление вопроса
        q_weights = self.self_attn(q_hidden, question_mask)
        #    return a weighted average of x (a sequence of vectors)
        q_output = q_weights.unsqueeze(1).bmm(q_hidden).squeeze(1)

        # 3) concat + linear
        q_output = q_output.unsqueeze(1)
        output = torch.cat([p_output, q_output], 1)
        output = output.mean(-1)
        output = self.lin(output)

        return output

In [288]:
model = DrQA_model(len(word_set))

lr=0.01
device_name="cpu"
device = torch.device(device_name)

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()
model = model.to(device)

logits = model(batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask) # Прямой проход
loss = loss_function(logits, batch_labels) # Подсчёт ошибки
loss.backward() # Подсчёт градиентов dL/dw
optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации

/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:39: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(y_mask.data, -float('inf'))
/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:69: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(x_mask.data, -float('inf'))


In [300]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cpu",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()#.cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask, batch_labels) in enumerate(get_next_gen_batch(train)):
            logits = model(batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask) # Прямой проход
            loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask, batch_labels) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask) # Прямой проход
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

In [301]:
model = DrQA_model(len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

Trainable params: 5501365


  0%|▏                                        | 32/7541 [00:04<16:32,  7.57it/s]/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:39: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(y_mask.data, -float('inf'))
/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:69: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(x_mask.data, -float('inf'))
7552it [23:03,  5.46it/s]                                                       
1888it [05:45,  5.46it/s]

Epoch = 0, Avg Train Loss = 1.5385, Avg val loss = 0.6367, Time = 1728.65s


In [320]:
def compute_precision_and_recall(true_positive, false_positive, false_negative):
    """
    Вычисляем точность и полноту по TP, FP и FN
    """
    if false_positive + true_positive > 0:
        precision = float(true_positive) / (true_positive + false_positive)
    else:
        precision = 0
    if false_negative + true_positive > 0:
        recall = float(true_positive) / (true_positive + false_negative)
    else:
        recall = 0
    return recall, precision

def predict(model, samples):
    model.eval()
    tp, fp, fn = 0,0,0
    for _, (indices, batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask, batch_labels) in enumerate(get_next_gen_batch(samples)):
        logits = model(batch_qst, batch_qst_mask, batch_psg, batch_psg_fts, batch_psg_mask)
        plabels = logits.max(dim=1)[1]
        tp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j!=0) and (i==j)])
        fp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j!=0) and (i!=j)])
        fn += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j==0) and (i!=j)])

    recall, precision = compute_precision_and_recall(tp, fp, fn)
    try:
        f_measure = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        f_measure = 'None'
    print('precision:', precision)
    print('recall:   ', recall)
    print('f1:       ', f_measure)

In [321]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------


  0%|▏                                        | 32/7541 [00:05<20:47,  6.02it/s]/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:39: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(y_mask.data, -float('inf'))
/var/folders/2q/qk755hxx3f5bjgcp14hnl93m_yy8wm/T/ipykernel_2698/2502503718.py:69: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1647.)
  scores.data.masked_fill_(x_mask.data, -float('inf'))
7552it [22:20,  5.63it/s]                                                       


precision: 0.7421614409606404
recall:    0.9502455690796497
f1:        0.8334113681056279

 -------on val dataset:--------


1888it [04:41,  6.70it/s]                                                       


precision: 0.6569435637285986
recall:    0.869857262804366
f1:        0.7485549132947977

 -------on test dataset:-------


3296it [07:59,  6.88it/s]                                                       

precision: 0.6549426138467235
recall:    0.8701426463354648
f1:        0.7473595268272074


## Часть 4. [3 балла] BiDAF-подобная архитектура

Основана на статье: Bidirectional Attention Flow for Machine Comprehension

Minjoon Seo, Aniruddha Kembhavi, Ali Farhadi, Hannaneh Hajishirzi

https://arxiv.org/abs/1611.01603

Архитектура BiDAF предложена для задачи SQuAD, но легко может быть адаптирована к текущему заданию. Модель состоит из следующих блоков:
1. Кодировщик  получает на вход два представления слова: эмбеддинг слова и полученное из CNN посимвольное представление слова. Кодировщики для вопроса и для параграфа одинаковы. 
2. Слой внимания (детальное описание приведено в статье, см. пункт Attention Flow Layer)
3. Промежуточный слой, который получает на вход контекстуализированные эмбеддинги слов из параграфа, состоящие из трех частей (выход кодировщика параграфа,   Query2Context (один вектор) и Context2Query (матрица) выравнивания

4. Слой предсказания. 

Предложите, как можно было модифицировать последний слой предсказания в архитектуре BiDAF, с учетом того, что итоговое предсказание – это метка yes / no, предсказание которой проще, чем предсказание спана ответа для SQuAD.

Оцените качество этой модели для решения задачи. 

[bonus] Замените входные эмбеддинги и все дополнительные признаки, используемые кодировщиками, на BERT эмбеддинги. Улучшит ли это качество результатов?

In [322]:
char_set = list( {ch for sample in train for token in sample.question for ch in token} |
                 {ch for sample in train for token in sample.passage for ch in token}
               )
char_set.insert(0, '<unk>'), char_set.insert(0, '<pad>')
len(char_set)

629

In [398]:
def get_next_gen_batch(samples, max_qst_len=50, max_psg_len=512, max_char_seq_len=40, batch_size=32):
    indices = np.arange(len(samples))
    np.random.shuffle(indices)
    batch_begin = 0
    with tqdm(total=len(samples)) as pbar:
        while batch_begin < len(samples):
            batch_indices = indices[batch_begin: batch_begin + 32]
            batch_qst = []
            batch_qst_ch = []
            batch_psg = []
            batch_psg_ch = []
            batch_labels = []
            for data_ind in batch_indices:
                ind = list(batch_indices).index(data_ind)
                sample = samples[data_ind]
                
                #вопрос
                question = torch.zeros(max_qst_len, dtype=torch.long) #.cuda()
                qst_chars = torch.zeros((max_qst_len, max_char_seq_len), dtype=torch.long)#.cuda()
                for token_num, token in enumerate(sample.question[:max_qst_len]):
                    #слова
                    question[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                    #символы
                    for char_num, char in enumerate(token[:max_char_seq_len]):
                        qst_chars[token_num][char_num] = char_set.index(char) if char in char_set else char_set.index('<unk>')
                
                #параграф
                passage = torch.zeros(max_psg_len, dtype=torch.long) #.cuda()
                psg_chars = torch.zeros((max_psg_len, max_char_seq_len), dtype=torch.long)#.cuda()
                for token_num, token in enumerate(sample.passage[:max_psg_len]):
                    #слова
                    passage[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                    #символы
                    for char_num, char in enumerate(token[:max_char_seq_len]):
                        psg_chars[token_num][char_num] = char_set.index(char) if char in char_set else char_set.index('<unk>')
                
                labels = sample.labels

                batch_qst.append(question)
                batch_qst_ch.append(qst_chars)
                batch_psg.append(passage)
                batch_psg_ch.append(psg_chars)
                batch_labels.append(labels)
              
            batch_begin += batch_size
            pbar.update(batch_size)
          
            batch_qst = torch.stack(batch_qst)
            batch_psg = torch.stack(batch_psg)
            batch_qst_ch = torch.stack(batch_qst_ch)
            batch_psg_ch = torch.stack(batch_psg_ch)
            batch_labels = torch.LongTensor(batch_labels)
            yield batch_indices, batch_qst, batch_qst_ch, batch_psg, batch_psg_ch, batch_labels

In [399]:
for _, batch_qst, batch_qst_ch, batch_psg, batch_psg_ch, batch_labels in get_next_gen_batch(train):
    1+1
    break
print()
print( 'batch_qst'.ljust(20), batch_qst.size() )
print( 'batch_qst_mask'.ljust(20), batch_qst_ch.size() )
print( 'batch_psg'.ljust(20), batch_psg.size() )
print( 'batch_psg_fts'.ljust(20), batch_psg_ch.size() )
print( 'batch_labels'.ljust(20), batch_labels.size() )

  0%|▏                                        | 32/7541 [00:04<17:20,  7.21it/s]


batch_qst            torch.Size([32, 50])
batch_qst_mask       torch.Size([32, 50, 40])
batch_psg            torch.Size([32, 512])
batch_psg_fts        torch.Size([32, 512, 40])
batch_labels         torch.Size([32])


In [416]:
class BiDAF_model(nn.Module):
    def __init__(self,
                 char_set_size, word_set_size,
                 question_max_size = 50, passage_max_size = 512,
                 char_embedding_dim=8, word_embedding_dim=64,
                 lstm_embedding_dim = 24, 
                 char_max_seq_len=40, kernel_size=3,
                 classes_count=2):
        super().__init__()
        
        self.char_embedding_dim = char_embedding_dim
        self.char_max_seq_len = char_max_seq_len
        self.char_embedding = nn.Embedding(char_set_size, char_embedding_dim)
        self.char_cnn = nn.Conv1d(in_channels=char_embedding_dim, out_channels=1, kernel_size=kernel_size)
        self.embedding = nn.Embedding(word_set_size, word_embedding_dim)
        self.context_LSTM = nn.LSTM(word_embedding_dim+char_max_seq_len-(kernel_size-1), lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.modeling_LSTM1 = nn.LSTM(8*lstm_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.modeling_LSTM2 = nn.LSTM(2*lstm_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.lin = nn.Linear(passage_max_size, classes_count)

        # Attention Flow Layer
        self.att_weight_c = nn.Linear(2*lstm_embedding_dim, 1)
        self.att_weight_q = nn.Linear(2*lstm_embedding_dim, 1)
        self.att_weight_cq = nn.Linear(2*lstm_embedding_dim, 1)
        
    def forward(self, questions, question_chars, passages, passage_chars):
        # https://github.com/galsang/BiDAF-pytorch/tree/master
        def att_flow_layer(c, q):
            """
            :param c: (batch, c_len, hidden_size * 2)
            :param q: (batch, q_len, hidden_size * 2)
            :return: (batch, c_len, q_len)
            """
            c_len = c.size(1)
            q_len = q.size(1)

            cq = []
            for i in range(q_len):
                #(batch, 1, hidden_size * 2)
                qi = q.select(1, i).unsqueeze(1)
                #(batch, c_len, 1)
                ci = self.att_weight_cq(c * qi).squeeze()
                cq.append(ci)
            # (batch, c_len, q_len)
            cq = torch.stack(cq, dim=-1)

            # (batch, c_len, q_len)
            s = self.att_weight_c(c).expand(-1, -1, q_len) + \
                self.att_weight_q(q).permute(0, 2, 1).expand(-1, c_len, -1) + \
                cq

            # (batch, c_len, q_len)
            a = F.softmax(s, dim=2)
            # (batch, c_len, q_len) * (batch, q_len, hidden_size * 2) -> (batch, c_len, hidden_size * 2)
            c2q_att = torch.bmm(a, q)
            # (batch, 1, c_len)
            b = F.softmax(torch.max(s, dim=2)[0], dim=1).unsqueeze(1)
            # (batch, 1, c_len) * (batch, c_len, hidden_size * 2) -> (batch, hidden_size * 2)
            q2c_att = torch.bmm(b, c).squeeze()
            # (batch, c_len, hidden_size * 2) (tiled)
            q2c_att = q2c_att.unsqueeze(1).expand(-1, c_len, -1)
            # q2c_att = torch.stack([q2c_att] * c_len, dim=1)

            # (batch, c_len, hidden_size * 8)
            x = torch.cat([c, c2q_att, c * c2q_att, c * q2c_att], dim=-1)
            return x
        
        # эмбеддинги слов w2v
        q_emb = self.embedding(questions)
        p_emb = self.embedding(passages)
        # полученное из CNN посимвольное представление слова
        q_c_emb = self.char_embedding(question_chars)
        q_c_emb = q_c_emb.reshape(q_c_emb.size(0)*q_c_emb.size(1), self.char_max_seq_len, self.char_embedding_dim)
        q_c_emb = q_c_emb.permute(0,2,1)
        q_c_cnn = self.char_cnn(q_c_emb)
        q_c_cnn = q_c_cnn.reshape(question_chars.size(0), question_chars.size(1), -1)
        #----
        p_c_emb = self.char_embedding(passage_chars)
        p_c_emb = p_c_emb.reshape(p_c_emb.size(0)*p_c_emb.size(1), self.char_max_seq_len, self.char_embedding_dim)
        p_c_emb = p_c_emb.permute(0,2,1)
        p_c_cnn = self.char_cnn(p_c_emb)
        p_c_cnn = p_c_cnn.reshape(passage_chars.size(0), passage_chars.size(1), -1)
        # объединение ембедингов слов и символов
        q = torch.cat([q_c_cnn, q_emb], dim=-1)
        p = torch.cat([p_c_cnn, p_emb], dim=-1)
        # контекстная рекуррентная сеть
        q = self.context_LSTM(q)[0]
        p = self.context_LSTM(p)[0]
        # Attention Flow Layer
        g = att_flow_layer(p, q)
        # LSTM modeling
        rnn1 = self.modeling_LSTM1(g)[0]
        rnn2 = self.modeling_LSTM2(rnn1)[0]
        # output
        output = rnn2.mean(-1)
        output = self.lin(output)
        return output

In [401]:
model = BiDAF_model(len(char_set), len(word_set))

lr=0.01
device_name="cpu"
device = torch.device(device_name)

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()
model = model.to(device)

logits = model(batch_qst, batch_qst_ch, batch_psg, batch_psg_ch) # Прямой проход
loss = loss_function(logits, batch_labels) # Подсчёт ошибки
loss.backward() # Подсчёт градиентов dL/dw
optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации

In [403]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cpu",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss()#.cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_qst, batch_qst_ch, batch_psg, batch_psg_ch, batch_labels) in enumerate(get_next_gen_batch(train)):
            logits = model(batch_qst, batch_qst_ch, batch_psg, batch_psg_ch) # Прямой проход
            loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_qst, batch_qst_ch, batch_psg, batch_psg_ch, batch_labels) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_qst, batch_qst_ch, batch_psg, batch_psg_ch) # Прямой проход
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

In [404]:
model = BiDAF_model(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

Trainable params: 2782102


7552it [25:51,  4.87it/s]                                                       
1888it [05:49,  5.41it/s]                                                       


Epoch = 0, Avg Train Loss = 1.6096, Avg val loss = 0.6590, Time = 1900.84s


In [406]:
def predict(model, samples):
    model.eval()
    tp, fp, fn = 0,0,0
    for _, (indices, batch_qst, batch_qst_ch, batch_psg, batch_psg_ch, batch_labels) in enumerate(get_next_gen_batch(samples)):
        logits = model(batch_qst, batch_qst_ch, batch_psg, batch_psg_ch)
        plabels = logits.max(dim=1)[1]
        tp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j!=0) and (i==j)])
        fp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j!=0) and (i!=j)])
        fn += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if (j==0) and (i!=j)])

    recall, precision = compute_precision_and_recall(tp, fp, fn)
    try:
        f_measure = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        f_measure = 'None'
    print('precision:', precision)
    print('recall:   ', recall)
    print('f1:       ', f_measure)

In [407]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------


7552it [19:54,  6.32it/s]                                                       


precision: 0.6210051717278875
recall:    1.0
f1:        0.7661976439790575

 -------on val dataset:--------


1888it [04:55,  6.38it/s]                                                       


precision: 0.6314952279957582
recall:    1.0
f1:        0.7741306467338316

 -------on test dataset:-------


3296it [08:39,  6.34it/s]                                                       

precision: 0.6217125382262997
recall:    1.0
f1:        0.7667358099189139


*модель не обучилась - всегда один ответ*

##### попытка №2 (BiDAF) -> smaller learning rate

In [420]:
model = BiDAF_model(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.001)

Trainable params: 2782102


7552it [23:15,  5.41it/s]                                                       
1888it [06:05,  5.16it/s]                                                       


Epoch = 0, Avg Train Loss = 1.5674, Avg val loss = 0.6579, Time = 1760.89s


In [421]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------


7552it [20:37,  6.10it/s]                                                       


precision: 0.6219090667375698
recall:    0.9989323083493488
f1:        0.7665710774272839

 -------on val dataset:--------


1888it [05:04,  6.21it/s]                                                       


precision: 0.6312997347480106
recall:    0.9991603694374476
f1:        0.7737321196358907

 -------on test dataset:-------


3296it [09:11,  5.97it/s]                                                       

precision: 0.6215554194733619
recall:    0.9985243482538121
f1:        0.7661822985468958


##### увы не помогло ☝️😔

Сравнение DrQA и BiDAF:
    
![](https://www.researchgate.net/profile/Felix_Wu6/publication/321069852/figure/fig1/AS:560800147881984@1510716582560/Schematic-layouts-of-the-BiDAF-left-and-DrQA-right-architectures-We-propose-to.png)

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

**1)** baseline = 0.6291 (fasttext на текстах, состоящих из склееных вопросов и абзацев) <br>
   низкое качество, поскольку архитектура в силу своей простоты не может выявить  <br>
   закономерности выборки  <br>
**2)** bert = 0.6067 <br>
   я бы сказал, что проблема та же - конкатенация эмбедингов абзаца и вопроса не  <br>
   позволяет учесть взаимосвязь между ними  <br>
**3)** DrQA = 0.6549 <br>
   возможно можно добавить механизм внимания выходами рекуррентных сетей над <br>
   вопросом и абзацем <br>
**4)** BiDAF = 0.6217 <br>
   модель не обучилась - в качестве предсказания выдает самый частый ответ выборки
   возможно можно обойтись одной рекуррентной сетью перед финальным линейным слоем
<br>
<br>
модель DrQA показала, что механизм внимания между абзацем и <br>
вопросом позвоялет лучше обучаться сети на предсказние ответа, <br>
однако в случае с BiDAF обучить сеть не удалось

In [ ]:
# urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip', 
#                            'pre-trained-fasttext.zip')

# from zipfile import ZipFile
# with ZipFile('pre-trained-fasttext.zip', 'r') as f:
#     f.extractall('pre-trained-fasttext')